## Predicting CRS scores
We now generate a pipeline that given a specific date, will forecast an estimated CRS cutoff value.

In [10]:
from utils import splitting as splt
from utils import preprocessing as prep

import mlflow
import pandas as pd
from mlflow.tracking import MlflowClient


In [11]:
query_date = '31-Jul-2024'

In [12]:
url = 'https://www.canada.ca/content/dam/ircc/documents/json/ee_rounds_123_en.json'
df = prep.create_df_from_website(url)
df = prep.cleanup_df_general_rounds(df)

# retrieve all the data prior to the query date
df_sub = df.loc[df.index < query_date].copy()


In [13]:
# create a new row just for prediction
df_query = df[:0].copy()
df_query.loc[query_date,'round type'] = 'General'

# set up dummy data
df_query.loc[query_date,'invitations issued'] = 0
df_query.loc[query_date,'CRS cutoff'] = 0


In [14]:
# concatenate to older data
df = pd.concat([df_query, df_sub])

In [15]:
# Calculate independent vars
df = prep.calculate_independent_vars(df)

In [27]:
# get only queried time 
x_labels=['dt-1','CRS-1']
y_labels=['CRS cutoff']
X_vals, y_vals = splt.create_features(df,x_labels,y_labels)

,dt-1,CRS-1
date,,
2024-07-31,99.0,529.0


In [ ]:
# get model
mlflow.set_tracking_uri("sqlite:///mlflow.db")
client = MlflowClient()

# get run id and set Model name
# run_id = best_performace_run.info.run_id
# model_uri = f"runs:/{run_id}/artifacts/model"

model_name = "CRS_Model"
model_version = 1

model = mlflow.pyfunc.load_model(model_uri=f"models:/{model_name}/{model_version}")

#model = mlflow.pyfunc.load_model(model_uri)

In [ ]:
# evaluate model given these X vals
y_vals['CRS cutoff'] = model.predict(X_vals)
y_vals



In [17]:
# roll_times=['30D','60D','90D','180D']

# # replace nans wit the closest available rolling average
# for i in range(1,len(roll_times)):
#     from_roll = f'roll_{roll_times[-i]}'
#     to_roll = f'roll_{roll_times[-i-1]}'

#     df[to_roll]= np.where(df[to_roll].isna(),df[from_roll],df[to_roll])


In [18]:
# df